# First Dataset

In [7]:
# import section
import pyomo.environ as pyo
import numpy as np
import pandas as pd 
from prettytable import PrettyTable

# config/data block
capacities = pd.read_csv('cap_d1.csv', index_col=0).to_numpy()
prof_preferences = pd.read_csv('prof_pref_d1.csv', index_col=0).to_numpy().transpose()
std_preferences = pd.read_csv('std_pref_d1.csv', index_col=0).to_numpy()
student_size = len(std_preferences[0])
prof_size = len(prof_preferences)

# sets
profs = list(range(prof_size))
students = list(range(student_size))


# parameters
def capacities_rule(model, i):
    return capacities[i,0]

def prof_preferences_rule(model, i, j):
    return prof_preferences[i,j]


def std_preferences_rule(model, i, j):
    return std_preferences[i,j]


#First objective function
def of1_rule(model):
  return sum(\
             ((model.prof_pref[i,j] + model.std_pref[i,j])\
             +( abs(model.prof_pref[i,j] - model.std_pref[i,j])+1))\
             *model.delta[i,j] for i in model.profs for j in model.students\
             )
 



#Second Objective Function
def of2_rule(model):
  prof_score = {}
  std_score = {}
  for i in model.profs:
      sum_prefs = sum(model.std_pref[i,j] for j in model.students)
      prof_score[i] = sum_prefs/prof_size
  for j in model.students:
      sum_prefs = sum(model.prof_pref[i,j] for i in model.profs)
      std_score[j] = sum_prefs/student_size

  return sum(\
                ((model.prof_pref[i,j] + model.std_pref[i,j])\
                +(abs(model.prof_pref[i,j] - model.std_pref[i,j]))\
                +(prof_score[i] + std_score[j])\
                +(abs(prof_score[i] - std_score[j])))\
                * model.delta[i,j] for i in model.profs for j in model.students\
              )
  

#Third Objective Function
def w_prof_pref(model,i,j):
  w_prof=1
  if model.prof_pref[i,j]>0.15*student_size:
    w_prof=1.5
  return w_prof

def w_std_pref(model,i,j):
  w_std = 1
  if model.std_pref[i,j]>0.15*prof_size:
    w_std=1.5
  return w_std

def of3_rule(model):
  prof_score = {}
  std_score = {}
  for i in model.profs:
      sum_prefs = sum(model.std_pref[i,j] for j in model.students)
      prof_score[i] = sum_prefs/prof_size
  for j in model.students:
      sum_prefs = sum(model.prof_pref[i,j] for i in model.profs)
      std_score[j] = sum_prefs/student_size
  return sum(\
  ((w_prof_pref(model,i,j)*model.prof_pref[i,j] + w_std_pref(model,i,j)*model.std_pref[i,j])\
                +(abs(model.prof_pref[i,j] - model.std_pref[i,j]) )\
                +(prof_score[i] + std_score[j])\
                +(abs(prof_score[i] - std_score[j])))\
                * model.delta[i,j] for i in model.profs for j in model.students\
              )


def of4_rule(model):
  return sum(model.u[i,j] +  model.prof_pref[i,j]*model.landa[i,j,j_prim]\
             for i in model.profs for j in model.students for j_prim in model.students)
  
#Hard Constraints
def single_prof_const_rule(model, j):
      return sum(model.delta[i,j] for i in model.profs) == 1
 
def prof_capacity_const_rule(model, i):
    return sum(model.delta[i,j] for j in model.students) == model.cap[i]

#Soft Constraints
def soft1_rule(model,i,j):
    return sum(model.delta[i_prim,j] for i_prim in model.profs if i_prim!= i and model.std_pref[i,j] < model.std_pref[i_prim,j] )  - model.u[i,j] == 0

def soft2_part1_rule(model,i,j,j_prim):
    M = 1
    if j_prim != j and model.prof_pref[i,j_prim] <= model.prof_pref[i,j] :
      return model.delta[i,j_prim] +\
            sum(model.delta[i_prim,j_prim] for i_prim in model.profs if i_prim!=i and model.prof_pref[i_prim,j_prim]<= model.prof_pref[i,j_prim])\
            + model.landa[i,j,j_prim]\
             >= model.delta[i,j]
    else:
       return pyo.Constraint.Skip

def solveProblem(model,i):
    #create Sets
    model.add_component('profs', pyo.Set(initialize=profs))
    model.add_component('students', pyo.Set(initialize=students))
    #create Parameters
    model.add_component('cap', pyo.Param(model.profs, initialize=capacities_rule))
    model.add_component('prof_pref', pyo.Param(model.profs, model.students, initialize=prof_preferences_rule))
    model.add_component('std_pref', pyo.Param(model.profs , model.students, initialize=std_preferences_rule))
    #variables
    model.add_component('delta', pyo.Var(model.profs, model.students, domain=pyo.Binary))

    #Objectives
    if i == 0:
      print("First Objective")
      model.add_component('objective_function', pyo.Objective(rule=of1_rule, sense=pyo.minimize))
    if i == 1:
      print("Second Objective")
      model.add_component('objective_function', pyo.Objective(rule=of2_rule, sense=pyo.minimize))
    if i==2:
      print("Third Objective")
      model.add_component('objective_function', pyo.Objective(rule=of3_rule, sense=pyo.minimize))
    if i==3:
      print("Fourth Objective")
      model.add_component('u', pyo.Var(model.profs, model.students, domain=pyo.NonNegativeReals))
      model.add_component('landa', pyo.Var(model.profs,model.students, model.students, domain=pyo.NonNegativeReals))
      model.add_component('objective_function', pyo.Objective(rule=of4_rule, sense=pyo.minimize))
      model.add_component('soft1', pyo.Constraint(model.profs, model.students, rule=soft1_rule))
      model.add_component('soft2_part1', pyo.Constraint(model.profs,model.students, model.students, rule=soft2_part1_rule))



    # constraints
    model.add_component('single_prof_per_student_const', pyo.Constraint(model.students, rule=single_prof_const_rule))
    model.add_component('prof_capacity_const', pyo.Constraint(model.profs, rule=prof_capacity_const_rule))
    # solver settings
    result=pyo.SolverFactory('glpk').solve(model, 'glpk')
    # model.pprint()
    # model.display()

    pairs_and_weights = []
    for i in model.profs:
      for j in model.students:
        if  pyo.value(model.delta[i,j])==1 : 
          pairs_and_weights.append((i,j))

    # Create table object
    table = PrettyTable()
    table.field_names = [""] + list(model.students)
    for i in model.profs:
        row_data = [i]
        for j in model.students:
            if (i, j) in [pair for pair in pairs_and_weights]:
                row_data.append((model.prof_pref[i,j],model.std_pref[i,j]))
            else:
                row_data.append("")
        table.add_row(list(row_data))
    # Print table
    print(table)


for i in range(4):
  model = pyo.ConcreteModel()
  solveProblem(model,i)

First Objective
+----+--------+--------+---------+--------+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+--------+--------+--------+----------+--------+--------+--------+--------+--------+--------+
|    |   0    |   1    |    2    |   3    |    4     |   5    |   6    |   7    |   8    |   9    |   10   |   11   |   12   |   13   |   14   |   15   |   16   |    17   |    18   |   19   |   20   |   21   |    22    |   23   |   24   |   25   |   26   |   27   |   28   |
+----+--------+--------+---------+--------+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+--------+--------+--------+----------+--------+--------+--------+--------+--------+--------+
| 0  |        |        |         |        |          |        |        |        |        |        |        |        |        |        |        |        |      

# Second Dataset

In [3]:

# import section
import pyomo.environ as pyo
import numpy as np
import pandas as pd
from prettytable import PrettyTable

# config/data block
capacities = pd.read_csv('cap_d2.csv', index_col=0).to_numpy()
prof_preferences = pd.read_csv('std_pref_d2.csv', index_col=0).to_numpy().transpose()
std_preferences = pd.read_csv('prof_pref_d2.csv', index_col=0).to_numpy()
student_size = len(std_preferences[0])
prof_size = len(prof_preferences)
# sets
profs = list(range(prof_size))
students = list(range(student_size))


# parameters
def capacities_rule(model, i):
    return capacities[i,0]

def prof_preferences_rule(model, i, j):
    return prof_preferences[i,j]


def std_preferences_rule(model, i, j):
    return std_preferences[i,j]


#First objective function
def of1_rule(model):
  return sum(\
             ((model.prof_pref[i,j] + model.std_pref[i,j])\
             +( abs(model.prof_pref[i,j] - model.std_pref[i,j])+1))\
             *model.delta[i,j] for i in model.profs for j in model.students\
             )
 



#Second Objective Function
def of2_rule(model):
  prof_score = {}
  std_score = {}
  for i in model.profs:
      sum_prefs = sum(model.std_pref[i,j] for j in model.students)
      prof_score[i] = sum_prefs/prof_size
  for j in model.students:
      sum_prefs = sum(model.prof_pref[i,j] for i in model.profs)
      std_score[j] = sum_prefs/student_size

  return sum(\
                ((model.prof_pref[i,j] + model.std_pref[i,j])\
                +(abs(model.prof_pref[i,j] - model.std_pref[i,j]))\
                +(prof_score[i] + std_score[j])\
                +(abs(prof_score[i] - std_score[j])))\
                * model.delta[i,j] for i in model.profs for j in model.students\
              )
  

#Third Objective Function
def w_prof_pref(model,i,j):
  w_prof=1
  if model.prof_pref[i,j]>0.15*student_size:
    w_prof=1.5
  return w_prof

def w_std_pref(model,i,j):
  w_std = 1
  if model.std_pref[i,j]>0.15*prof_size:
    w_std=1.5
  return w_std

def of3_rule(model):
  prof_score = {}
  std_score = {}
  for i in model.profs:
      sum_prefs = sum(model.std_pref[i,j] for j in model.students)
      prof_score[i] = sum_prefs/prof_size
  for j in model.students:
      sum_prefs = sum(model.prof_pref[i,j] for i in model.profs)
      std_score[j] = sum_prefs/student_size
  return sum(\
  ((w_prof_pref(model,i,j)*model.prof_pref[i,j] + w_std_pref(model,i,j)*model.std_pref[i,j])\
                +(abs(model.prof_pref[i,j] - model.std_pref[i,j]) )\
                +(prof_score[i] + std_score[j])\
                +(abs(prof_score[i] - std_score[j])))\
                * model.delta[i,j] for i in model.profs for j in model.students\
              )

#Fourth Objective Func
def of4_rule(model):
  return sum(model.u[i,j] + prof_preferences[i,j]*model.landa[i,j,j_prim]\
             for i in model.profs for j in model.students for j_prim in model.students)
  

#Soft Constraints
def soft1_rule(model,i,j):
    return sum(model.delta[i_prim,j] for i_prim in model.profs if i_prim!= i and model.std_pref[i,j] < model.std_pref[i_prim,j] )  - model.u[i,j] == 0

def soft2_part1_rule(model,i,j,j_prim):
    M = 1
    if j_prim != j and model.prof_pref[i,j_prim] <= model.prof_pref[i,j] :
      return model.delta[i,j_prim] +\
            sum(model.delta[i_prim,j_prim] for i_prim in model.profs if i_prim!=i and model.prof_pref[i_prim,j_prim]<= model.prof_pref[i,j_prim])\
            + model.landa[i,j,j_prim]\
             >= model.delta[i,j]
    else:
       return pyo.Constraint.Skip

#Hard Constraints
def single_prof_const_rule(model, j):
      return sum(model.delta[i,j] for i in model.profs) == 1
 
def prof_capacity_const_rule(model, i):
    return sum(model.delta[i,j] for j in model.students) == model.cap[i]

def solveProblem(model,i):
    #create Sets
    model.add_component('profs', pyo.Set(initialize=profs))
    model.add_component('students', pyo.Set(initialize=students))
    #create Parameters
    model.add_component('cap', pyo.Param(model.profs, initialize=capacities_rule))
    model.add_component('prof_pref', pyo.Param(model.profs, model.students, initialize=prof_preferences_rule))
    model.add_component('std_pref', pyo.Param(model.profs , model.students, initialize=std_preferences_rule))
    #variables
    model.add_component('delta', pyo.Var(model.profs, model.students, domain=pyo.Binary))

    #Objectives
    if i == 0:
      print("First Objective")
      model.add_component('objective_function', pyo.Objective(rule=of1_rule, sense=pyo.minimize))
    if i == 1:
      print("Second Objective")
      model.add_component('objective_function', pyo.Objective(rule=of2_rule, sense=pyo.minimize))
    if i==2:
      print("Third Objective")
      model.add_component('objective_function', pyo.Objective(rule=of3_rule, sense=pyo.minimize))
    if i==3:
      print("Fourth Objective")
      model.add_component('u', pyo.Var(model.profs, model.students, domain=pyo.NonNegativeReals))
      model.add_component('landa', pyo.Var(model.profs,model.students, model.students, domain=pyo.NonNegativeReals))
      model.add_component('objective_function', pyo.Objective(rule=of4_rule, sense=pyo.minimize))
      model.add_component('soft1', pyo.Constraint(model.profs, model.students, rule=soft1_rule))
      model.add_component('soft2_part1', pyo.Constraint(model.profs,model.students, model.students, rule=soft2_part1_rule))


    # constraints
    model.add_component('single_prof_per_student_const', pyo.Constraint(model.students, rule=single_prof_const_rule))
    model.add_component('prof_capacity_const', pyo.Constraint(model.profs, rule=prof_capacity_const_rule))
    # solver settings
    result=pyo.SolverFactory('glpk').solve(model, 'glpk')
    # model.pprint()
    # model.display()

    
    pairs_and_weights = []

    for i in model.profs:
      for j in model.students:
        if  pyo.value(model.delta[i,j])==1 : 
          pairs_and_weights.append((i,j))

    # Create table object
    table = PrettyTable()
    # Add columns to table
    table.field_names = [""] + list(model.students)
    # Add rows to table
    for i in model.profs:
        row_data = [i]
        for j in model.students:
            if (i, j) in [pair for pair in pairs_and_weights]:
                row_data.append((model.prof_pref[i,j],model.std_pref[i,j]))
            else:
                row_data.append("")
        table.add_row(list(row_data))
    # Print table
    print(table)


for i in range(4):
  model = pyo.ConcreteModel()
  solveProblem(model,i)

First Objective
+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+--------+--------+--------+--------+--------+--------+
|    |   0    |   1    |   2    |   3    |   4    |   5    |   6    |   7    |   8    |    9    |   10   |   11   |   12   |   13   |   14   |   15   |
+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+--------+--------+--------+--------+--------+--------+
| 0  |        |        |        |        |        |        |        |        |        |         |        |        |        | (1, 7) |        |        |
| 1  |        |        |        |        |        |        |        |        |        |         |        |        | (2, 1) |        |        |        |
| 2  |        |        | (1, 1) |        |        |        |        |        |        |         |        |        |        |        |        |        |
| 3  |        |        |        |        |        |        |        |   